## 埋込モデル

### 準備

必要なライブラリのインストール

In [ ]:
%pip install -r ../requirements.txt

### 埋め込みモデル


In [1]:
from langchain_cohere import CohereEmbeddings

embeddings = CohereEmbeddings(
    model="embed-multilingual-v3.0"
)

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


### Document Loader

`../docs/*.txt` に格納されているテキストデータを読み込み、LangChain の Document へ変換します。

In [2]:
import glob
from langchain.document_loaders import TextLoader

files = glob.glob("../docs/*.txt")
documents = []

for file in files:
    loader = TextLoader(file_path=file)
    document = loader.load()
    documents.extend(document)
documents

[Document(metadata={'source': '../docs/arani.txt'}, page_content='BB流！200円で作れる真鯛のアラの塩煮の作り方\n\n用意するもの\n\n- フライパン\n- アルミホイル\n- しょうが1欠片\n- 真鯛のアラ（血とかは洗っておく）\n- 塩\n- 水\n\n作り方は以下の通りです。\n\n1．フライパンに水（落とし蓋をするのでやや少なめ）を生姜のスライスと一緒に入れる\n2．沸騰したら塩を小さじ2（スープがしょっぱかったので少なくてもいいかも）を入れて溶かしてから真鯛のアラを入れる。沸騰してから入れると前処理無しで臭みなし\n3．アルミホイルで落とし蓋する\n4．火が通るまで弱火で適当に煮る\n5．(゜Д゜)ｳﾏｰ\n\nおつまみにぴったりです。ぜひ作ってみてね。\n'),
 Document(metadata={'source': '../docs/butakimuchi.txt'}, page_content='BB流！安くて早くて旨い男のための豚キムチの作り方\n\n材料は以下の通りです。\n\n- 豚ロース薄切り100g\n- キムチ100g\n- タマネギ1/2個（レタス少量などでも可）\n- ごま油\n- 酒\n- 醤油\n\n作り方は以下の通りです。\n\n1. 豚ロース1枚を4分割くらいに切る\n2. タマネギを適当に切る\n3. フライパンにごま油を適当に引いて豚ロースを焼く、タマネギも投入\n4. 肉の色が変わったらキムチ投入、酒と醤油小さじ1ずつ入れて混ぜて焼いて終わり\n\n簡単で白米に合うこと間違いなし！ぜひ作ってみてね\n'),
 Document(metadata={'source': '../docs/torimomo-shiokouji.txt'}, page_content='BB流！鶏ももの塩麹漬け焼きの作り方です。晩御飯にぴったりだね。\n\n材料\n- 鶏モモ（別に何でも。鮭とか鯛の切り身でも可）\n- 塩麹\n\n作り方は以下の通りです。\n\n1. 鶏モモは厚みを揃え（めんどくさければそのまま）、ジップロックとかビニール袋に投入\n2. 塩麹を両面に塗りつける\n3. 冷蔵庫に投入して半日～1日そのまま。朝にやって夜に食べるとか前の晩にやっておくと

### Text Splitters

In [3]:
from langchain_community.document_loaders import WebBaseLoader

langchain_doc = WebBaseLoader("https://python.langchain.com/docs/concepts/").load()

len(langchain_doc[0].page_content)


USER_AGENT environment variable not set, consider setting it to identify your requests.


15540

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

langchain_docs = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(langchain_doc)
len(langchain_docs)

23

In [5]:
from langchain_community.vectorstores import FAISS

langchain_db = FAISS.from_documents(langchain_docs, CohereEmbeddings(model="embed-multilingual-v3.0"))
result = langchain_db.similarity_search("Please tell me how to set up the PR to separate it into minor and patch versions ?")
result

[Document(metadata={'source': 'https://python.langchain.com/docs/concepts/', 'title': 'Conceptual guide | 🦜️🔗 LangChain', 'description': 'This guide provides explanations of the key concepts behind the LangChain framework and AI applications more broadly.', 'language': 'en'}, page_content='middle" effectHow to split Markdown by HeadersHow to merge consecutive messages of the same typeHow to add message historyHow to migrate from legacy LangChain agents to LangGraphHow to retrieve using multiple vectors per documentHow to pass multimodal data directly to modelsHow to use multimodal promptsHow to create a custom Output ParserHow to use the output-fixing parserHow to parse JSON outputHow to retry when a parsing error occursHow to parse XML outputHow to parse YAML outputHow to use the Parent Document RetrieverHow to use LangChain with different Pydantic versionsHow to add chat historyHow to get a RAG application to add citationsHow to do per-user retrievalHow to get your RAG application to

### Vector Store

自然言語を用いた類似度検索を行います。

In [6]:
vector_store = FAISS.from_documents(documents, embeddings)
result = vector_store.similarity_search(query="カルビクッパ",k=1)
result

[Document(metadata={'source': '../docs/karubikuppa.txt'}, page_content='BB流カルビクッパの作り方です。\n\nまずは、以下の材料を用意します。なお、材料は2人分です。\n\n- 牛肉: 50g\n- 大根: 半分\n- 人参: 1本\n- シイタケ: 2-3個\n- ニンニク: 半分\n- もやし: 1袋\n- ニラ: 半分\n- ぜんまい: 40g\n    - BB流ポイント 「入れなくても良いですが、入れると@<ttb>{めちゃ旨}です！」\n- つゆ (*)\n    - 水: 800cc\n    - コチュジャン: 大さじ2\n        - BB流ポイント「入れれば入れるだけ美味しい！」\n    - ウェイパー: 大さじ2\n    - ごま油: 大さじ2\n        - BB流ポイント「入れれば入れるだけ美味しい！」\n    - 酒: 大さじ2\n    - 醤油: 大さじ2\n    - 砂糖: 大さじ2\n- 塩コショウ:適量\n- 卵: 2個\n\nごま油でニンニクを炒め、香りが出たら牛肉を炒めます。つゆ（*）をいれ、ニラともやし以外を入れてひと煮立ちさせます。\n最後にニラともやしを入れてある程度シャキシャキ感を残しつつ、卵をふんわりさせたらBB流カルビクッパの完成です！\nなお、卵をふんわりさせるコツはつゆを限界まで沸騰させることです！\n')]

### Retriever

Vector StoreからRetrieverの生成

In [7]:
retriever = vector_store.as_retriever(search_kwargs={"k": 1})
retriever.invoke("カルビクッパ")

[Document(metadata={'source': '../docs/karubikuppa.txt'}, page_content='BB流カルビクッパの作り方です。\n\nまずは、以下の材料を用意します。なお、材料は2人分です。\n\n- 牛肉: 50g\n- 大根: 半分\n- 人参: 1本\n- シイタケ: 2-3個\n- ニンニク: 半分\n- もやし: 1袋\n- ニラ: 半分\n- ぜんまい: 40g\n    - BB流ポイント 「入れなくても良いですが、入れると@<ttb>{めちゃ旨}です！」\n- つゆ (*)\n    - 水: 800cc\n    - コチュジャン: 大さじ2\n        - BB流ポイント「入れれば入れるだけ美味しい！」\n    - ウェイパー: 大さじ2\n    - ごま油: 大さじ2\n        - BB流ポイント「入れれば入れるだけ美味しい！」\n    - 酒: 大さじ2\n    - 醤油: 大さじ2\n    - 砂糖: 大さじ2\n- 塩コショウ:適量\n- 卵: 2個\n\nごま油でニンニクを炒め、香りが出たら牛肉を炒めます。つゆ（*）をいれ、ニラともやし以外を入れてひと煮立ちさせます。\n最後にニラともやしを入れてある程度シャキシャキ感を残しつつ、卵をふんわりさせたらBB流カルビクッパの完成です！\nなお、卵をふんわりさせるコツはつゆを限界まで沸騰させることです！\n')]

WikipediaRetriever

In [8]:
from langchain_community.retrievers import WikipediaRetriever

retriever = WikipediaRetriever()
docs = retriever.invoke("Mobile Suit Gundam SEED Freedom")
docs[0].page_content[:50]

'Mobile Suit Gundam SEED Freedom (Japanese: 機動戦士ガンダ'

### BB流のレシピを教えてくれるアプリケーション

In [9]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

from langchain_cohere.chat_models import ChatCohere

chat = ChatCohere(
    model="command-r-plus"
)
retriever = vector_store.as_retriever()

prompt_template = PromptTemplate.from_template(template="""
あなたは有能なアシスタントです。
以下のコンテキストに基づいて質問に対する回答を作成してください。
分からない場合は分からないと回答してください。

## コンテキスト

{context}

## 質問

{question}
""")

chain = (
    {"question": RunnablePassthrough(), "context": retriever}
    | prompt_template
    | chat
    | StrOutputParser()
)

response = chain.stream("カルビクッパってどうやって作るのでしょうか？")

for chunk in response:
    print(chunk, end="")


カルビクッパの作り方は次のとおりです。

**材料（2人分）**

- 牛肉 50g
- 大根 半分
- 人参 1本
- シイタケ 2~3個
- ニンニク 半分
- モヤシ 1袋
- ニラ 半分
- ゼンマイ 40g （オプション）
- 水 800cc
- コチュジャン 大さじ2
- ウェイパー 大さじ2
- ゴマ油 大さじ2
- 酒 大さじ2
- 醤油 大さじ2
- 砂糖 大さじ2
- 塩コショウ 適量
- 卵 2個

**作り方**

1. ゴマ油でニンニクを炒め、香りが出たら牛肉を炒めます。
2. つゆ（水、コチュジャン、ウェイパー、ゴマ油、酒、醤油、砂糖を混ぜたもの）を入れます。
3. ニラともやし以外の材料を入れ、ひと煮立ちさせます。
4. 最後にニラともやしを入れてシャキシャキ感を残しつつ、卵をふんわりとさせます。つゆを限界まで沸騰させると卵がふんわり仕上がります。

ポイント：ゼンマイを入れると一層美味しくなります。また、コチュジャン、ゴマ油、ウェイパーは多めに入れるとより美味しくなるそうです。

以上でBB流カルビクッパの完成です！

### xQL完全ガイドについて教えてくれるアプリケーション

In [10]:
xql_doc = WebBaseLoader("https://techbookfest.org/product/vwEgK9fAmzRphNukv4E83P?productVariantID=b6iAh0AVyEs4hCUczPiy").load()
xql_db = FAISS.from_documents(xql_doc, CohereEmbeddings(model="embed-multilingual-v3.0"))
xql_retriever = xql_db.as_retriever()

In [11]:

chain = (
    {"question": RunnablePassthrough(), "context": xql_retriever}
    | prompt_template
    | chat
    | StrOutputParser()
)

response = chain.stream("xQL完全ガイドってなに？")

for chunk in response:
    print(chunk, end="")


xQL完全ガイドは、PromQL、LogQL、およびTraceQL という3つのクエリ言語に焦点を当てたガイドブックです。これらの言語は、Prometheus、Grafana Loki、およびGrafana Tempo などのオープンソースのモニタリングツールで使用されます。このガイドでは、これらのクエリ言語をマスターすることで、ユーザーが収集したテレメトリ データから必要なメトリクス、ログ、トレースをより簡単に見つけられるようにするのが目的です。

このガイドには、メトリクス、ログ、トレースの概要、各クエリ言語の説明、練習問題、およびサンプル アプリと Docker 環境が含まれており、ユーザーが実際にクエリを実行できるようになっています。このガイドを通じて、著者は読者が PromQL、LogQL、および TraceQL をより深く理解し、クラウドネイティブ アプリケーションの運用に効果的に活用できるようになることを願っています。